# Scraping roguelike articles on the web

This Python notebook describes the data collection process for roguelike universe. If you have not yet installed the requirements, you can do it by running:

`pip install -r requirements.txt`

## Sourcing game titles

We prepared a [list of roguelike games](https://en.wikipedia.org/wiki/List_of_roguelikes) from Wikipedia as a starting point. For indicators of out-of-genre influences, we sourced 10,000+ video game titles from Pastebin as uploaded by the user ________. 

First, we setup read/write functions that are friendly with international unicode characters, because the web may contain all kinds of character code points.

In [75]:
import os
import io
import json
import pandas as pd

def read_json(path):
    data = ''
    with io.open(path, 'r', encoding='utf-8') as f:
        data = json.loads(f.read())
        print(__message('Loaded {}'.format(path)))
    return data
    
def save_json(path, data):
    with io.open(path, 'w', encoding='utf-8') as f:
        try:
            output = json.dumps(data, indent=2, ensure_ascii=False)
            f.write(output)
        except UnicodeEncodeError:
            f.write(output.encode('utf-8'))
    print(__message('Written to {}'.format(path)))
    
def __success(text):
    return '  (SUCC) {}'.format(text).encode('utf-8')
    
def __failure(text):
    return '!!FAIL!! {}'.format(text).encode('utf-8')
    
def __warning(text):
    return '??WARN?? {}'.format(text).encode('utf-8')
    
def __message(text):
    return '   |MSG| {}'.format(text).encode('utf-8')

Here is a sample of the list of roguelike games from Wikipedia:

In [66]:
roguelikes = pd.read_csv(os.path.join(os.getcwd(), 'roguelikes.csv'), skip_blank_lines=True)
roguelikes.head()

,Name,RogueTemple,Link,Released,Updated,Developer,Theme,Influences
0,100 Rogues,http://roguebasin.roguelikedevelopment.org/ind...,http://www.100rogues.com/,2010/05/06,2010/05/06,Dinofarm Games,Fantasy,Rogue
1,1Quest,http://roguebasin.roguelikedevelopment.org/ind...,http://www.ratzngodz.fr,2014/02/20,2015/02/07,Ratz 'N' Godz,Fantasy,"DCSS, Dominion 4"
2,3059,http://roguebasin.roguelikedevelopment.org/ind...,https://sites.google.com/site/free3069/3059---...,2005/00/00,2005/06/11,Phr00t,"Science Fiction, Alien Planets, Futuristic",nethack
3,3069,http://roguebasin.roguelikedevelopment.org/ind...,http://sites.google.com/site/free3069/,2009/07/06,2009/10/06,Phr00t,"Science Fiction, Alien Planets, Futuristic",3059
4,3079,http://roguebasin.roguelikedevelopment.org/ind...,http://sites.google.com/site/3079game/,2011/10/25,2015/02/13,Phr00t,"Science Fiction, Alien Planets, Futuristic","3059, 3069, Fallout, Minecraft"


In addition, roguelike-like games:

In [82]:
roguelikelikes = pd.read_csv(os.path.join(os.getcwd(), 'roguelike-likes.csv'), skip_blank_lines=True)
roguelikelikes.head()

,Name,Released,Updated,Developer,Theme,Influences
0,ToeJam & Earl,1991,NaN,Johnson Voorsanger Productions,Fantasy,NaN
1,Diablo,1996,NaN,Blizzard North,Fantasy,NaN
2,Diablo II,2000,NaN,Blizzard Entertainment,Fantasy,NaN
3,Lost Labyrinth,2001,2011.0,Lost Labyrinth,Fantasy,NaN
4,Strange Adventures In Infinite Space,2002,2004.0,"Rich Carlson, Iikka Keränen",Space science fiction,NaN


And a sample of the list of video games:

In [52]:
video_games = pd.read_json(os.path.join(os.getcwd(), 'games.json'))
video_games.head(10)

,title,year
0,$hop-n-$pree,2009
1,'43 - One Year After,1986
2,'89 Denno Kyusei Uranai,1988
3,'Nam 1965-1975,1991
4,'Splosion Man,2009
5,'Til Death Do Us Part,2013
6,(Almost) Total Mayhem,2011
7,(Not) Just another Space Shooter,2004
8,(T)Raumschiff Surprise - Periode 1,2004
9,*NSYNC Hotline Phone and Fantasy CD-Rom Game,2001


## Building a corpus 

Before we can do any text analysis, we need to build a corpus in which to operate on.

### 1. RogueTemple

RogueTemple Wiki collects a detailed description of roguelike games.

In [76]:
import requests

def scrape_mediawiki_url(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.84 Safari/537.36',
    }
    response = requests.get(url, headers=headers, timeout=(9.1, 12.1))
    soup = bs4.BeautifulSoup(response.text, 'lxml')
    
    content = [node.text.strip() for node in soup.select('#mw-content-text') if node.text]
    return ''.join(content)

In [ ]:
# Sample scrape
content = scrape_mediawiki_url('http://roguebasin.roguelikedevelopment.org/index.php?title=100_Rogues')
print(content)

### 2. Wikipedia

Searching on Wikipedia can be done with two dozens line of code.

In [77]:
import wikipedia

def scrape_wiki_id(pageid):
    page = wikipedia.page(pageid=pageid)    
    print_wiki_page(page)
    
def scrape_wiki(title):
    try:
        searchstring = title
        page = wikipedia.page(searchstring, auto_suggest=False)
#         print_wiki_page(page)
        return page
    except wikipedia.DisambiguationError:
        try:
            searchstring = '{} (video game)'.format(title).replace(' ', '_')
            page = wikipedia.page(searchstring, auto_suggest=False)
#             print_wiki_page(page)
            return page
        except wikipedia.DisambiguationError:
            try:
                searchstring = '{} (Unix video game)'.format(title).replace(' ', '_')
                page = wikipedia.page(searchstring, auto_suggest=False)
    #             print_wiki_page(page)
                return page
            except:
                print(__warning(u'Wikipedia cannot find "{}"'.format(searchstring)))
        except:
            print(__warning(u'Wikipedia cannot find "{}"'.format(searchstring)))
    except wikipedia.PageError:
        try:
            page = wikipedia.page(title, auto_suggest=False)
#             print_wiki_page(page)
            return page
        except:
            print(__warning(u'Search term "{}" returned nothing'.format(searchstring)))
    
def print_wiki_page(page):
    print(page.title)
    print(page.content)
    print(page.references)  

In [78]:
# Test Wikipedia crawl
print(scrape_wiki('Rogue Legacy'))

<WikipediaPage 'Rogue Legacy'>


### 3. DuckDuckGo

We also source a list of potential interesting webpages via an internet search engine, DuckDuckGo.

In [79]:
import bs4
import time
import requests
import urllib.parse

def scrape_duckduckgo(keywords, developer=""):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.84 Safari/537.36',
    }
    searchstring = u'"{}" AND {} AND game AND (interview OR mortem OR history OR develop)'.format(keywords, developer)
    q = u'http://duckduckgo.com/html/?q={}'.format(urllib.parse.quote(searchstring.encode('utf-8')))
    print(q)
                                                   
    response = requests.get(q, headers=headers, timeout=(9.1, 12.1))
    soup = bs4.BeautifulSoup(response.text, 'lxml')
    
    links = []
    links = [node.get('href') for node in soup.select('a.result__a')]
    return links

In [63]:
scrape_duckduckgo('Ancient Domains of Mystery', 'Thomas Biskup')

http://duckduckgo.com/html/?q=%22Ancient%20Domains%20of%20Mystery%22%20AND%20Thomas%20Biskup%20AND%20game%20AND%20%28interview%20OR%20mortem%20OR%20history%20OR%20develop%29


['https://store.steampowered.com/video/333300',
 'https://lgdb.org/game/adom-ancient-domains-of-mystery',
 'https://en.wikipedia.org/wiki/Ancient_Domains_of_Mystery',
 'https://strategywiki.org/wiki/Ancient_Domains_of_Mystery',
 'https://www.ranker.com/review/ancient-domains-of-mystery/455750',
 'https://ancient-domains-of-mystery.ru.uptodown.com/windows/download',
 'https://www.rockpapershotgun.com/tag/adom-ancient-domains-of-mystery/',
 'https://www.cultureofgaming.com/ancient-domains-of-mystery-adom-review/',
 'https://www.youtube.com/watch?v=ChtBuBrFYc8',
 'http://RuTracker.org/forum/viewtopic.php?t=5451914',
 'https://classicreload.com/ancient-domains-of-mystery.html',
 'https://www.facebook.com/ADOMAncientDomainsOfMystery/',
 'https://steamcommunity.com/sharedfiles/filedetails/?l=russian&id=258925365',
 'https://alchetron.com/Ancient-Domains-of-Mystery',
 'https://www.turkaramamotoru.com/en/ancient-domains-of-mystery-181815.html',
 'https://www.giantbomb.com/ancient-domains-of-my

## Scrape the corpus

With the functions above we can collect a corpus.

In [80]:
def scrape(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.84 Safari/537.36',
    }
    try:
        print(__message('Scraping {} ...'.format(url)))
        response = requests.get(url, headers=headers, timeout=(9.1, 12.1))
    except Exception as e:
        print(__failure('Failed to load {}'.format(url)))
        print(e)
        return None
    
    html = response.text
    if html and any(word in html.lower() for word in ['tutorial']):
        return None
    if html and any(word in html.lower() for word in ['rogue', 'procedural', 'generation', 'interview', 'mortem', 'review', 'history', 'develop', 'idea', 'inspir']):
        print(__message('Found article'))
        soup = bs4.BeautifulSoup(response.text, 'lxml')
        selections = soup.select('body > p') + soup.select('div > p') + soup.select('table td')
        content = [node.text.strip() for node in selections]
        return ''.join(content)
    return None

In [71]:
# For Roguelike games, we build a corpus with RogueTemple and DuckDuckGo
corpus = []
    
for index, roguelike in roguelikes.iterrows():
    print(roguelike)
    title = roguelike['Name']
    if not isinstance(title, str):
        continue
    text = []
    
    rogue_temple = scrape_mediawiki_url(roguelike['RogueTemple'])
    text.append(rogue_temple)

    developers = str(roguelike['Developer']).replace(',', ' OR ')
    links = scrape_duckduckgo(title, developers)
    
    for link in links[:10]:
        if 'roguebasin.roguelikedevelopment.org' in link \
            or 'roguebasin.com' in link \
            or 'wikipedia' in link:
            continue
        content = scrape(link)
        if content:
            text.append(content)
    
    corpus.append({"title": title, "text": text})
  
save_json('corpus.json', corpus)

b'   |MSG| Loaded corpus.json'
Name                                                  100 Rogues
RogueTemple    http://roguebasin.roguelikedevelopment.org/ind...
Link                                   http://www.100rogues.com/
Released                                              2010/05/06
Updated                                               2010/05/06
Developer                                         Dinofarm Games
Theme                                                    Fantasy
Influences                                                 Rogue
Name: 0, dtype: object
http://duckduckgo.com/html/?q=%22100%20Rogues%22%20AND%20Dinofarm%20Games%20AND%20game%20AND%20%28interview%20OR%20mortem%20OR%20history%20OR%20develop%29
b'   |MSG| Scraping http://www.dinofarmgames.com/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://100rogues.com/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.facebook.com/100-Rogues-109776630046/ ...'
b'   |MSG| Scraping https://www.patreon.c

http://duckduckgo.com/html/?q=%223089%22%20AND%20Phr00t%20AND%20game%20AND%20%28interview%20OR%20mortem%20OR%20history%20OR%20develop%29
b'   |MSG| Scraping https://www.youtube.com/watch?v=atp6rqHoAec ...'
b'   |MSG| Scraping https://store.steampowered.com/video/263360?l=czech ...'
b'   |MSG| Scraping http://small-games.info/?go=game&c=4&i=12408 ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://steamcommunity.com/app/414510/discussions/0/458606877314643131/ ...'
b'   |MSG| Scraping https://translate.google.ru/ ...'
b'   |MSG| Scraping https://indiegamereviewer.com/review-3089-an-rpg-roguelike-shooter-by-indie-dev-phr00t/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://twitter.com/phr00t_/status/951280635439669253 ...'
b'   |MSG| Scraping https://www.reddit.com/r/3089/ ...'
b'   |MSG| Scraping http://3089game.wikia.com/ ...'
b'   |MSG| Scraping https://www.facebook.com/IndieGameHq/?ref=py_c ...'
Name                                                      7KBRLL
RogueTe

b'   |MSG| Scraping https://www.thisisbarry.com/single-post/2016/10/26/Donnie-Darko-2001-Full-Plot-and-Ending-Explained ...'
Name                                                    Alphaman
RogueTemple    http://roguebasin.roguelikedevelopment.org/ind...
Link                                                         NaN
Released                                              1995/00/00
Updated                                               1995/00/00
Developer                                       Jeffrey R. Olson
Theme                                           Post-Apocalyptic
Influences                                                   NaN
Name: 11, dtype: object
http://duckduckgo.com/html/?q=%22Alphaman%22%20AND%20Jeffrey%20R.%20Olson%20AND%20game%20AND%20%28interview%20OR%20mortem%20OR%20history%20OR%20develop%29
b'   |MSG| Scraping https://archive.org/details/Alphaman11 ...'
b'   |MSG| Scraping https://www.youtube.com/watch?v=6y_U3ra7TfA ...'
b'   |MSG| Scraping https://www.lds.org/gen

b'   |MSG| Scraping https://www.choiceofgames.com/2016/11/end-game-and-victory-design/ ...'
b'   |MSG| Scraping https://www.coursera.org/lecture/gamification/1-5-history-of-gamification-7Wp4p ...'
Name                         Anoxic Depths: Caves of the Yendori
RogueTemple    http://roguebasin.roguelikedevelopment.org/ind...
Link                    http://studiotectorum.com/anoxic-depths/
Released                                              2015/06/03
Updated                                               2015/08/16
Developer                             Studio Tectorum (Tectorum)
Theme                                          Underwater, SCUBA
Influences                                         Anoxic Depths
Name: 17, dtype: object
http://duckduckgo.com/html/?q=%22Anoxic%20Depths%3A%20Caves%20of%20the%20Yendori%22%20AND%20Studio%20Tectorum%20%28Tectorum%29%20AND%20game%20AND%20%28interview%20OR%20mortem%20OR%20history%20OR%20develop%29
b'   |MSG| Scraping https://www.youtube.com/watch?v

b'   |MSG| Scraping https://www.reddit.com/r/roguelikes/comments/79im83/break_out_of_harekas_underground_version_04/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://github.com/anaseto/boohu/blob/master/README.md ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://archive.org/details/boohu-0.5 ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.reddit.com/r/roguelikedev/comments/70mhce/i_wrote_a_short_roguelike_named_boohu_in_go_and_i/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://linuxfr.org/ ...'
b'   |MSG| Scraping https://linuxfr.org/news ...'
Name                                                        BOSS
RogueTemple    http://roguebasin.roguelikedevelopment.org/ind...
Link           https://web.archive.org/web/20130118205257/htt...
Released                                              1990/11/00
Updated                                               2012/10/12
Developer                           Robert Gulledge, Jason Black
Theme          

...

http://duckduckgo.com/html/?q=%22Ularn%22%20AND%20Phil%20Cordier%20OR%20%20Josh%20Brandt%20OR%20%20Josh%20Bressers%20OR%20%20David%20Richerby%20AND%20game%20AND%20%28interview%20OR%20mortem%20OR%20history%20OR%20develop%29
b'   |MSG| Scraping https://translate.google.ru/ ...'
b'   |MSG| Scraping http://larn.org/history/history.html ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://ru-ru.facebook.com/public/Josh-Brandt ...'
b'   |MSG| Scraping https://twitter.com/joshbrandt ...'
b'   |MSG| Scraping https://www.job-interview-site.com/chef-interview-questions-and-answers.html ...'
b'   |MSG| Scraping https://www.youtube.com/watch?v=TzCay7qcPyM ...'
b'   |MSG| Scraping https://www.ranker.com/review/josh-hutcherson/1326575 ...'
b'   |MSG| Scraping https://github.com/joshbressers/ularn ...'
b'   |MSG| Scraping https://teachershelp.ru/verbickaya-m-v-forward-angliyskiy-yazyk-dlya-6-kl-2/ ...'
Name                                                  UltraRogue
RogueTemple    http://rogueb

b'   |MSG| Scraping http://quiz-questions.net/ ...'
b'   |MSG| Scraping https://conversationstartersworld.com/21-questions-game/ ...'
Name                                                UnReal World
RogueTemple    http://roguebasin.roguelikedevelopment.org/ind...
Link                                   http://www.unrealworld.fi
Released                                              1992/00/00
Updated                                               2018/04/01
Developer                                          Sami Maaranen
Theme                                                   Survival
Influences                                                   NaN
Name: 243, dtype: object
http://duckduckgo.com/html/?q=%22UnReal%20World%22%20AND%20Sami%20Maaranen%20AND%20game%20AND%20%28interview%20OR%20mortem%20OR%20history%20OR%20develop%29
b'   |MSG| Scraping https://www.youtube.com/watch?v=Fibxbx31LUs ...'
b'   |MSG| Scraping http://www.unrealworld.fi/urw_reviews.html ...'
b'   |MSG| Found article'
b'

b'   |MSG| Scraping https://apkpure.com/web-raid-mobile/de.agsteiner.android.webraid ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.youtube.com/watch?v=Mfo6qx_tNbA ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://apkga.com/de.agsteiner.android.webraid ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://www.brothergames.com/android/web_raid_mobile-150198.html ...'
b'   |MSG| Scraping https://sameapk.com/web-raid-mobile/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://androidappsapk.co/detail-web-raid-mobile/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.similarplay.com/karlheinz_agsteiner/web_raid_mobile/apps/de.agsteiner.android.webraid ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://1apk.co/products-info/web-raid-mobile-apk-file ...'
Name                                                   WindTales
RogueTemple    http://roguebasin.roguelikedevelopment.org/ind...
Link           http://www.windowsphone.com/en-us/store/

b'   |MSG| Scraping https://www.twinfinite.net/2015/05/the-witcher-3-wild-hunt-guide-how-to-make-world-state-decisions/ ...'
b'   |MSG| Scraping http://mossmouth.com/forums/index.php?topic=1555.0;wap2 ...'
b'   |MSG| Scraping https://www.9news.com.au/national/2018/08/06/10/14/sydney-man-admits-foreign-fighting-charge ...'
b'   |MSG| Scraping http://collider.com/game-of-thrones-animated-history-seven-kingdoms/ ...'
b'   |MSG| Scraping http://www.triviaquestionsnow.com/for/sports-trivia ...'
b'   |MSG| Scraping https://gamesdeveloping.com/ ...'
Name                                                       XLarn
RogueTemple    http://roguebasin.roguelikedevelopment.org/ind...
Link                   http://swinfjord-games.com/LarnII-us.html
Released                                              2015/05/25
Updated                                               2016/07/30
Developer                                        Swinfjord-Games
Theme                                                    Fant

b'   |MSG| Scraping http://www.statemaster.com/encyclopedia/Topi-Ylinen ...'
b'   |MSG| Found article'
Name                                                    ZigClimb
RogueTemple    http://roguebasin.roguelikedevelopment.org/ind...
Link                                            http://flend.net
Released                                              2010/01/10
Updated                                               1000/00/00
Developer                                                  flend
Theme                                                      Egypt
Influences                                     A Journey to Hell
Name: 259, dtype: object
http://duckduckgo.com/html/?q=%22ZigClimb%22%20AND%20flend%20AND%20game%20AND%20%28interview%20OR%20mortem%20OR%20history%20OR%20develop%29
b'   |MSG| Scraping http://flend.net/zigclimb/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://translate.google.ru/ ...'
b'   |MSG| Scraping https://www.youtube.com/watch?v=9jiYciZ7Npk ...'
b'   |MSG| 

In [83]:
# For Roguelike-like games, we build a corpus with Wikipedia and DuckDuckGo
# corpus = read_json('corpus-roguelike-like.json')
# if not corpus:
corpus = []
    
for index, roguelike in roguelikelikes.iterrows():
    print(roguelike)
    title = roguelike['Name']
    text = []
    
    page = scrape_wiki(title)
    if page:
        text.append(page.content)

    developers = str(roguelike['Developer']).replace(',', ' OR ')
    links = scrape_duckduckgo(title, developers)
    
    for link in links[:20]:
        if 'roguebasin.roguelikedevelopment.org' in link \
            or 'roguebasin.com' in link \
            or 'wikipedia' in link:
            continue
        content = scrape(link)
        if content:
            text.append(content)
    
    corpus.append({"title": title, "text": text})
  
save_json('corpus-roguelike-like.json', corpus)

Name                           ToeJam & Earl
Released                                1991
Updated                                  NaN
Developer     Johnson Voorsanger Productions
Theme                                Fantasy
Influences                               NaN
Name: 0, dtype: object
http://duckduckgo.com/html/?q=%22ToeJam%20%26%20Earl%22%20AND%20Johnson%20Voorsanger%20Productions%20AND%20game%20AND%20%28interview%20OR%20mortem%20OR%20history%20OR%20develop%29
b'   |MSG| Scraping http://gaming.wikia.com/wiki/ToeJam_%26_Earl_Productions ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://segaretro.org/ToeJam_%26_Earl ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.facebook.com/pages/ToeJam-Earl-Productions/109640125728425 ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://toejamandearl.wikia.com/wiki/ToeJam_%26_Earl_in_Panic_on_Funkotron ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.sega-16.com/2005/02/history-of-toejam-earl/ ...'
b'

/usr/local/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


http://duckduckgo.com/html/?q=%22Diablo%22%20AND%20Blizzard%20North%20AND%20game%20AND%20%28interview%20OR%20mortem%20OR%20history%20OR%20develop%29
b'   |MSG| Scraping https://us.battle.net/forums/en/d3/topic/7415795753 ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.diabloii.net/blog/comments/blizzard-north-didnt-know-diablo-is-story-ending ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.diablowiki.net/Blizzard_North ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://topasnew.club/6645-diablo-1996-istoriya-sozdaniya-igry-sozdanie-blizzard-north.html ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.youtube.com/watch?v=PJDMJaJbRtQ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.gamespot.com/articles/new-diablo-game-project-in-development-blizzard-co/1100-6459406/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://diablo.gamepedia.com/Blizzard_Entertainment ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https:

b'   |MSG| Scraping https://strange-adventures-in-infinite-space.en.uptodown.com/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://www.ag.ru/games/strange-adventures-in-infinite-space/review ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://www.digital-eel.com/sais/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.youtube.com/watch?v=B5IcX0gY3Js ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.myabandonware.com/game/strange-adventures-in-infinite-space-3qy ...'
b'   |MSG| Scraping https://strange-adventures-in-infinite-space.soft112.com/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.tor.com/2018/08/07/book-reviews-an-informal-history-of-the-hugos-by-jo-walton/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://www.g4g.it/2009/10/01/strange-adventures-in-infinite-space/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.gamespot.com/reviews/strange-adventures-in-infinite-space-review/1900-2856375/ ...'
b'  

b'   |MSG| Found article'
b'   |MSG| Scraping http://therutor.org/torrent/432207/the-binding-of-isaac-v1.0r10-2011-pc/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://streamers.gamepedia.com/The_Binding_of_Isaac ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://Torrent-Games.net/news/the_binding_of_isaac_edmund_mcmillen_eng/2011-09-29-20262 ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.polygon.com/features/2018/5/15/17345246/the-past-present-and-future-of-the-binding-of-isaac ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.pcgame.com/the-binding-of-isaac ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://bestgamer.net/load/3125-the-binding-of-isaac-edmund-mcmillen-eng-p.html ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://RuTracker.org/forum/viewtopic.php?t=3773360 ...'
b'   |MSG| Scraping https://steamcommunity.com/workshop/filedetails/?id=895960454 ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://edmundmcmil

...

b'   |MSG| Found article'
b'   |MSG| Scraping https://steamcommunity.com/app/242680/discussions/0/1696049513769195793/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.japantimes.co.jp/news/2018/08/06/national/ican-champions-grass-roots-efforts-persuade-japan-others-support-nuclear-free-world/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.gameskinny.com/google-amp/ybob1/nuclear-throne-review-fish-has-gills-and-i-have-no-skills ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.youtube.com/watch?v=VFTL5T-6Cdc ...'
b'   |MSG| Scraping http://devgamm-talks.com/rami-ismail-nuclear-throne/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.facebook.com/NuclearThroneGame/ ...'
b'   |MSG| Scraping https://waypoint.vice.com/en_us/article/kwzgje/read-an-excerpt-from-120-years-of-vlambeer-and-friends ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://yal.cc/nuclear-throne-u99/ ...'
b'   |MSG| Scraping http://ocean0fgames.com/nuclear-

b'   |MSG| Scraping https://www.gamepressure.com/games/runestone-keeper/z94ac1 ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://freegogpcgames.com/815/runestone-keeper/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.runestonekeeper.com/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://pcgamingwiki.com/wiki/Runestone_Keeper ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://gamefaqs.gamespot.com/pc/137653-runestone-keeper ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://store.steampowered.com/app/339400/Runestone_Keeper/ ...'
b'   |MSG| Scraping https://www.microsoft.com/en-us/p/runestonekeeper/9nn1r85tq39s ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://twitter.com/runestonekeeper ...'
b'   |MSG| Found article'
Name          Skyshine's Bedlam
Released                   2015
Updated                     NaN
Developer        Skyshine Games
Theme          Post-apocalyptic
Influences                  NaN
Name: 28, dtype: object

http://duckduckgo.com/html/?q=%22Enter%20the%20Gungeon%22%20AND%20Dodge%20Roll%20AND%20game%20AND%20%28interview%20OR%20mortem%20OR%20history%20OR%20develop%29
b'   |MSG| Scraping https://twitter.com/dodgerollgames ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://enterthegungeon.gamepedia.com/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://dodgeroll.com/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://gameranx.com/features/id/48447/article/enter-the-gungeon-interview-with-dodge-rolls-dave-crooks-the-past-present-and-future/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.reddit.com/r/EnterTheGungeon/ ...'
b'   |MSG| Scraping https://steamcommunity.com/app/311690/discussions/0/135507548127470722/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.trueachievements.com/game/Enter-the-Gungeon ...'
b'   |MSG| Scraping https://www.pcgamer.com/enter-the-gungeon-review/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://oceanofga

b'   |MSG| Found article'
b'   |MSG| Scraping https://pediaview.com/openpedia/List_of_roguelikes ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://www.revolvy.com/topic/Chronology%20of%20roguelike%20video%20games&item_type=topic ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://m.localwiki.org/ann-arbor/_explore/list ...'
b'!!FAIL!! Failed to load https://m.localwiki.org/ann-arbor/_explore/list'
HTTPSConnectionPool(host='m.localwiki.org', port=443): Read timed out. (read timeout=12.1)
b'   |MSG| Scraping https://vdocuments.mx/documents/ad-75-2-the-1970s-is-here-and-now.html ...'
b'   |MSG| Scraping https://www.manualtolyf.com/2014/04/joel-lambert-tries-to-evade-philippine.html ...'
b'   |MSG| Scraping https://keithtopping.blogspot.nl/2014/10/i-wasnt-there-i-watched-it-on-telly.html ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://allevents.in/vienna/2016-08-21 ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://vdocuments.site/documents/move-commercia

b'   |MSG| Found article'
b'   |MSG| Scraping https://www.baseball-reference.com/players/r/roberbr01.shtml ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://your-teachers.ru/anglijskij/testy-ege/grammatika-3/01/test-20-po-ege-anglijskij-yazyk-otvet ...'
b'   |MSG| Scraping https://www.job-interview-site.com/what-is-your-greatest-achievement-interview-question-and-answers.html ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://bgr.com/2015/12/14/comcast-ceo-brian-roberts-interview/ ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://znanija.com/task/20070252 ...'
b'   |MSG| Found article'
b'   |MSG| Scraping https://targetjobs.co.uk/careers-advice/interview-questions/398212-what-is-your-most-significant-achievement-tricky-graduate-interview-question ...'
b'   |MSG| Found article'
b'   |MSG| Scraping http://tonail.com/%D1%87%D1%82%D0%B5%D0%BD%D0%B8%D0%B5-%D0%B2-%D1%84%D0%BE%D1%80%D0%BC%D0%B0%D1%82%D0%B5-%D0%B5%D0%B3%D1%8D-%D0%B2%D0%B0%D1%80%D0%B8%D0%B0%D0%BD%D1%82

In [24]:
import io
import os
import re
import bs4
import sys
import json
import time
import nltk
import urllib
import pprint
import random
import string
import requests
import wikipedia
import itertools
import collections

In [ ]:
# Extract themes
# Items of interest, of genre, of identification
# Emotions of joy, sadness, frustrations
# Memory recall? Specific sentices or sentiments
corpus = read_json(os.path.join(os.getcwd(), 'data', 'corpus.json'))
distributions = {}

for game, sites in corpus.items():
    print(game)
    tagged_sentences = []
    for url, content in sites.items():
        for sentence in content:
            tagged_sentences += (encode_english(sentence))
    freqdist = nltk.FreqDist((word, tag) for word, tag in tagged_sentences if tag == u'ADJ')
    distributions[game] = freqdist

In [ ]:
for game, dist in distributions.items():
    words = [x[0] for x, y in distributions[game].most_common(10)]
    print(u'{}: {}'.format(game, u', '.join(words)))

In [ ]:
# Testing
scrape_wiki(u"Dungeon_(video_game)")

In [ ]:
# Scrape for links
game_meta = read_json(os.path.join(os.getcwd(), 'data', 'game-sources.json'))

shuffled_game_meta = game_meta.items()
random.shuffle(shuffled_game_meta)
for game, meta in shuffled_game_meta:
    game_meta[game]['Links'] += scrape_duckduckgo(game, game_meta[game]['Developer'])
    game_meta[game]['Links'] = list(set(game_meta[game]['Links']))
    save_json(os.path.join(os.getcwd(), 'data', 'game-sources.json'), game_meta)
    time.sleep(2)

In [ ]:
# Load content in search results
game_meta = read_json(os.path.join(os.getcwd(), 'data', 'game-sources.json'))
cached = read_json(os.path.join(os.getcwd(), 'data', 'corpus.json'))

output = cached
for game, meta in game_meta.items():
    if game not in output:
        output[game] = {}
    print(__message(game))
    for url in meta['Links']:
        if url in output[game] or url.endswith('pdf'):
            continue
        data = []
        html = scrape(url)
        if html and any(word in html.lower() for word in ['interview', 'mortem', 'review', 'history', 'develop', 'idea', 'inspir']):
            soup = bs4.BeautifulSoup(html)
            content = soup.select('div > p') + soup.select('body > p')
            data = [c.string.strip() for c in content if c.string and c.string.strip()]
            output[game][url] = data
            print(__message(u'Scrapped {}'.format(url)))
            save_json(os.path.join(os.getcwd(), 'data', 'corpus.json'), output)

In [ ]:
# Locate mentions of games
game_LUT = set(read_json(os.path.join(os.getcwd(), 'data', 'games.json')))
game_meta = read_json(os.path.join(os.getcwd(), 'data', 'game-sources.json'))
game_articles = read_json(os.path.join(os.getcwd(), 'data', 'corpus.json'))
not_games = set(read_json(os.path.join(os.getcwd(), 'data', 'not-games.json')))

# Create a look up table for games
roguelike_LUT = {}
for game, meta in game_meta.items():
    roguelike_LUT[game] = game
    if 'AKA' in meta:
        for aka in meta['AKA']:
            roguelike_LUT[aka] = game

In [ ]:
# Look through the interview articles
roguelike_relations = {}
other_relations = {}
for game, articles in game_articles.items():
    roguelike_relations[game] = []
    other_relations[game] = []
    counter = collections.Counter()
    for url, article in articles.items():
        # Intersection for fast search
        things = []
        current = u''
        for paragraph in article:
            for token in paragraph.split():
                if re.compile("^[A-Z0-9][\w:']*[\w:']|[A-Z\.]+$").match(token) or \
                        (current and token in ('the', 'of', 'no', 'to')):
                    current += u'{} '.format(token)
                elif current:
                    things.append(current.strip())
                    current = u''
        roguelike_things = [roguelike_LUT[s] for s in things if s in roguelike_LUT]
        if roguelike_things:
            roguelike_relations[game].extend(roguelike_things)
        other_things = [s for s in things if
                            s in game_LUT and
                            s not in not_games and
                            s not in roguelike_LUT and
                            len(s) > 1 and
                            not s.isdigit()]
        if other_things:
            other_relations[game].extend(other_things)

# print("\n### ROGUELIKES ###\n")
# pprint.pprint(roguelike_relations, indent=2)
# print("\n### OTHER GAMES ###\n")
# pprint.pprint(other_relations, indent=2)

save_json(os.path.join(os.getcwd(), 'generated', 'roguelike-relations.json'), roguelike_relations)
save_json(os.path.join(os.getcwd(), 'generated', 'other-relations.json'), other_relations)

In [ ]:
# Construct influence network

roguelike_relations = read_json(os.path.join(os.getcwd(), 'generated', 'roguelike-relations.json'))
other_relations = read_json(os.path.join(os.getcwd(), 'generated', 'other-relations.json'))
games_years = read_json(os.path.join(os.getcwd(), 'generated', 'games-years.json'))

roguelike_influence = {}
for roguelike, other_roguelikes in roguelike_relations.items():
    roguelike_influence[roguelike] = []
    
    roguelike_relation_counter = collections.Counter()
    for other_roguelike in other_roguelikes:
        if other_roguelike != roguelike:
            roguelike_relation_counter[other_roguelike] += 1
            
    other_relation_counter = collections.Counter()
    for other_relation in other_relations[roguelike]:
        if other_relation != roguelike:
            other_relation_counter[other_relation] += 1
            
    for roguelike_relation in roguelike_relation_counter.most_common(5):
        roguelike_influence[roguelike].append(roguelike_relation[0])
        
    for other_relation in other_relation_counter.most_common(5):
        if other_relation[1] > 1:
            roguelike_influence[roguelike].append(other_relation[0])
            
#     print(u'{}\n{}\n{}\n'.format(roguelike, 
#                                    roguelike_relation_counter.most_common(3), 
#                                    other_relation_counter.most_common(3)))

games_set_small = set(itertools.chain(*(roguelike_relations.values()+other_relations.values())))
    
games_years_small = {game: int(year) for game, year in games_years.items() if game in games_set_small}
    
print(games_years_small)
                                                        
save_json(os.path.join(os.getcwd(), 'generated', 'relations.json'), roguelike_influence)
save_json(os.path.join(os.getcwd(), 'generated', 'games-years-small.json'), games_years_small)